#  Fraud Detection use XGBoost with Graph Features

This notebook will demonstate using cuDF for ETL/data cleaning and XGBoost for training a fraud predection model.  
The processing will not use an additional graph processing or GNNs

In [1]:
# Since we are adding graph, need to import cuGraph
import cugraph

In [2]:
# requiered imports
import cudf
import cuml
import xgboost as xgb
import math

/home/brad/anaconda3/envs/cugraph_dev/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Let's only look at the Graph part for now - the rest will be similar to what we have already done

In [3]:
# base directoty
base_dir = "./elliptic_bitcoin_dataset/"

In [4]:
df_edges = cudf.read_csv(base_dir + "elliptic_txs_edgelist.csv")

In [5]:
df_edges.head(5)

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [6]:
# does the data start at 0?
(df_edges['txId1'].min(), df_edges['txId2'].min())

(1076, 1076)

no - that means that we need to use the renumbering feature of cuGraph to create
a contiguous sequence

In [7]:
G = cugraph.Graph(directed=False)

In [8]:
G.from_cudf_edgelist(df_edges, source='txId1', destination='txId2', renumber=True)

In [9]:
print(f"number of nodes: {G.number_of_nodes()}")
print(f"number of edges: {G.number_of_edges()}")

number of nodes: 203769
number of edges: 234355


In [10]:
# that is a very odd graph
# let's look at degree centrality

In [11]:
degree = G.degrees()

In [12]:
degree.head(5)

,in_degree,out_degree,vertex
0,10,10,97309263
1,14,14,98486557
2,10,10,99218169
3,10,10,102343697
4,9,9,105431244


In [13]:
degree.describe()

,in_degree,out_degree,vertex
count,203769.000000,203769.000000,2.037690e+05
mean,2.300203,2.300203,1.711310e+08
std,4.328377,4.328377,1.104655e+08
min,1.000000,1.000000,1.076000e+03
25%,1.000000,1.000000,8.433452e+07
50%,2.000000,2.000000,1.624375e+08
75%,2.000000,2.000000,2.454798e+08
max,473.000000,473.000000,4.032446e+08


In [14]:
# this is a good indication that there are disjoint subgraphs in the data

In [15]:
comp = cugraph.weakly_connected_components(G)

In [16]:
# Use groupby on the 'labels' column of the WCC output to get the counts of each connected component with the same label
label_count = comp.groupby('labels').count()
label_count.rename(columns={"vertex": "count"}, inplace=True)

print("Total number of components found : ", "{:,}".format(len(label_count)))

Total number of components found :  49


In [17]:
# The fact that there are multiple components impact the creationg of metrics.  Many algorithm normalize results
# based on the size of data, but for good metrics the size should be that of the component.  

# let's ignore the fact thgat there are multiple components and not normalize

In [18]:
bc = cugraph.betweenness_centrality(G, k=1000, normalized=False)

In [19]:
pr = cugraph.pagerank(G)
hits = cugraph.hits(G, normalized=False)
katz = cugraph.katz_centrality(G, normalized=False)

In [20]:
# merge all the data together

In [21]:
graph_metrics = degree.merge(pr, on="vertex")
graph_metrics = graph_metrics.merge(bc ,on="vertex")
graph_metrics = graph_metrics.merge(hits ,on="vertex")
graph_metrics = graph_metrics.merge(katz ,on="vertex")


In [22]:
# Now rename "vertex" to be "txId"
graph_metrics.rename(columns={'vertex' : 'txId'}, inplace=True)

### The XGBoost part

### Data Loading

In [23]:
# read the data files
df_features = cudf.read_csv(base_dir + 'elliptic_txs_features.csv', header=None)
df_classes  = cudf.read_csv(base_dir + "elliptic_txs_classes.csv")

In [24]:
# replace the value and set the type to int32
df_classes['class'] = df_classes['class'].replace("unknown", "0").astype('int32')

### merge the classes into the feature dataset
but we might need to adjust the dataframe some

In [25]:
# change the column 0 name to be txId to match the classes dataframe
df_features.rename(columns={'0' : 'txId'}, inplace=True)

In [26]:
# merging dataframes
df_merge = df_features.merge(df_classes, how='left', on='txId')

In [27]:
# Now add  the graph features
df_merge = df_merge.merge(graph_metrics, how='left', on='txId')

### Pull out the labeled data into groups for training, validating, and testing

In [28]:
classified   = df_merge.loc[df_merge['class'] != 0]
unclassified = df_merge.loc[df_merge['class'] == 0]

In [29]:
classified.head(5)

,txId,1,2,3,4,5,6,7,8,9,...,165,166,class,in_degree,out_degree,pagerank,betweenness_centrality,hubs,authorities,katz_centrality
1,121298347,6,0.136187,-0.184626,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.152681,...,-1.760926,-1.760984,2,1,1,0.000003,0.00,3.339366e-58,5.292198e-58,0.002212
3,121655492,6,-0.172834,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163494,...,-0.120613,-0.119792,2,1,1,0.000002,0.00,3.181162e-52,4.012152e-52,0.002212
9,9676808,6,-0.006516,-0.132897,-1.201369,-0.121970,0.015676,-0.113002,-0.061584,0.006676,...,-0.865922,-0.776269,2,1,1,0.000003,0.00,4.101941e-57,6.545955e-57,0.002212
13,8986809,6,-0.155460,-0.184668,-1.201369,0.103143,-0.063725,0.138585,-0.061584,-0.160758,...,-0.120613,-0.119792,2,1,1,0.000003,0.00,3.190602e-62,6.581474e-62,0.002212
28,121298641,6,0.069303,-0.184626,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.084249,...,1.519700,1.521399,2,2,2,0.000004,91696.05,6.415145e-55,1.035925e-54,0.002215


In [30]:
# reset the index 
classified.reset_index(inplace=True, drop=True)

In [31]:
classified.head(5)

,txId,1,2,3,4,5,6,7,8,9,...,165,166,class,in_degree,out_degree,pagerank,betweenness_centrality,hubs,authorities,katz_centrality
0,121298347,6,0.136187,-0.184626,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.152681,...,-1.760926,-1.760984,2,1,1,0.000003,0.00,3.339366e-58,5.292198e-58,0.002212
1,121655492,6,-0.172834,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163494,...,-0.120613,-0.119792,2,1,1,0.000002,0.00,3.181162e-52,4.012152e-52,0.002212
2,9676808,6,-0.006516,-0.132897,-1.201369,-0.121970,0.015676,-0.113002,-0.061584,0.006676,...,-0.865922,-0.776269,2,1,1,0.000003,0.00,4.101941e-57,6.545955e-57,0.002212
3,8986809,6,-0.155460,-0.184668,-1.201369,0.103143,-0.063725,0.138585,-0.061584,-0.160758,...,-0.120613,-0.119792,2,1,1,0.000003,0.00,3.190602e-62,6.581474e-62,0.002212
4,121298641,6,0.069303,-0.184626,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.084249,...,1.519700,1.521399,2,2,2,0.000004,91696.05,6.415145e-55,1.035925e-54,0.002215


### Split data into training and validation sets
cuML has a nice function for doing this

In [32]:
X_train, X_test = cuml.model_selection.train_test_split(classified, test_size=0.3, random_state=0)

In [33]:
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)

In [34]:
X_train['class'].value_counts()

2    29401
1     3194
Name: class, dtype: int32

In [35]:
X_test['class'].value_counts()

2    12618
1     1351
Name: class, dtype: int32

In [36]:
# Pull out the class column and then drop from th etraining set
Y_train = X_train[['class']]
X_train.drop(columns=['class'], inplace=True)

In [37]:
Y_test = X_test[['class']]
X_test.drop(columns=['class'], inplace=True)

### Use XGBoost

In [38]:
# Create a DMatrix
dtrain = xgb.DMatrix(X_train, Y_train)

In [39]:
# Train XGBoost
params = {
    'learning_rate'  : 0.3,
    'max_depth'      : 8,
    'objective'      : 'reg:squarederror',
    'subsample'      : 0.6,
    'gamma'          : 1,
    'silent'         : True,
    'verbose_eval'   : True,
    'tree_method'    :'gpu_hist'
}


In [40]:
trained_model = xgb.train(params, dtrain)

[13:11:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117836726/work/src/learner.cc:576: 
Parameters: { "silent", "verbose_eval" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [41]:
# test
dtest = xgb.DMatrix(X_test, Y_test)

In [42]:
Y_test['prediction'] = trained_model.predict(dtest)

In [43]:
Y_test['squared_error'] = (Y_test['prediction'] - Y_test['class'])**2

In [44]:
Y_test.head()

,class,prediction,squared_error
0,2,1.958971,0.001683
1,2,1.954335,0.002085
2,2,1.917087,0.006875
3,2,1.931744,0.004659
4,2,1.962319,0.001420


In [45]:
Y_test[Y_test['class'] == 1].head()

,class,prediction,squared_error
8,1,0.990539,0.000090
15,1,0.990539,0.000090
28,1,0.990539,0.000090
29,1,1.789822,0.623819
33,1,1.901209,0.812178


In [46]:
# compute the actual RMSE over the full test set
RMSE = Y_test['squared_error'].mean()
math.sqrt(RMSE)

0.1220505355291152

In [47]:
Y_test[Y_test['prediction'] > 1.5]['class'].value_counts()

2    12602
1      191
Name: class, dtype: int32